In [94]:
from configparser import *

In [95]:
from snowflake.snowpark import Session
import snowflake.snowpark.functions as F
from datetime import date

In [96]:
import sys, json
with open("connection_parameters_true.json") as jsonfile:
    credentials_dict = json.load(jsonfile)

In [97]:
def snowpark_session_create():
    session = Session.builder.configs(credentials_dict).create()
    return session

In [98]:
demo_session = snowpark_session_create()

In [99]:
df = demo_session.sql('select * from snowflake_sample_data.tpcds_sf10tcl.item limit 1000')

In [100]:
#df.show()

In [101]:
#for row in df.collect():
#    print(row.I_ITEM_ID)

In [102]:
demo_session.use_database("snowflake_sample_data")
demo_session.use_schema("tpcds_sf10tcl")

In [103]:
df = demo_session.table("item")

df = df.select(
    "I_ITEM_ID",
    "I_REC_START_DATE",
    "I_REC_END_DATE",
    "I_ITEM_DESC",
    "I_CURRENT_PRICE",
    "I_WHOLESALE_COST",
    "I_CATEGORY",
    "I_BRAND",  
).filter((F.col("I_REC_START_DATE").between(date(1999,1,1),date(2010,12,31)))&(F.col("I_REC_START_DATE").is_not_null())&(F.col("I_REC_END_DATE").is_not_null()))

In [ ]:
df.show()

In [105]:
df = df.with_column("Margin",(F.col("I_WHOLESALE_COST") - (F.col("I_CURRENT_PRICE"))))

In [ ]:
df.sort(F.col("Margin").desc()).show()

In [107]:
total1999 = df.group_by("I_CATEGORY").agg(F.sum("Margin").alias("Margin_1999"))

In [ ]:
total1999.show()

In [109]:
df2 = demo_session.table("item")

df2 = df.select(
    "I_ITEM_ID",
    "I_REC_START_DATE",
    "I_REC_END_DATE",
    "I_ITEM_DESC",
    "I_CURRENT_PRICE",
    "I_WHOLESALE_COST",
    "I_CATEGORY",
    "I_BRAND",  
).filter((F.col("I_REC_START_DATE").between(date(2000,1,1),date(2001,12,31))))

df2 = df2.with_column("Margin",(F.col("I_WHOLESALE_COST") - (F.col("I_CURRENT_PRICE"))))

total2001 = df.group_by("I_CATEGORY").agg(F.sum("Margin").alias("Margin_2001"))

In [110]:
total1999 = total1999.with_column_renamed(F.col("Margin_1999"),"Margin_1999")

In [111]:
joinnedDFs = total1999.join(total2001, total1999.I_CATEGORY == total2001.I_CATEGORY).select(
    total1999.col("I_CATEGORY").alias("I_CATEGORY"),
    total1999.col("Margin_1999").alias("Margin_1999"),
    total2001.col("Margin_2001").alias("Margin_2001"),
)

In [112]:
joinnedDFs = (
    joinnedDFs.with_column(
        "Totals",
        F.array_construct(F.col("Margin_1999"),F.col("Margin_2001"))
    )
    .with_column(
        "Comment",
        F.concat(F.lit("Logic was created by "), F.current_user())
    )
    .with_column(
        "TRUST_LEVEL",
        F.when(F.col("I_CATEGORY") == "Sports", 0.4)
        .otherwise(1)
    )
)

In [114]:
joinnedDFs = joinnedDFs.drop(F.col("Margin_1999")).drop(F.col("Margin_2001"))

In [115]:
demo_session.close()